## Introduction:

This notebook is devoted to for applying ANN as a function approximation .. 

The source of differentiation of this notebook is that it utilizes SKlearn library instead of  building NN from scratch or even using deep learning .. 


Link of ANN guide: https://www.machinelearningtutorial.net/2017/01/28/python-scikit-simple-function-approximation/

Link of the dataset: https://www.kaggle.com/pankajjsh06/ibm-watson-marketing-customer-value-data

Data exploration guides for the same dataset: 

https://www.kaggle.com/dktalaicha/predict-customer-life-time-value-clv

https://www.kaggle.com/sreshta140/analysis-on-response-and-clv

In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [85]:
df = pd.read_csv('CLV.csv')
df.head()

,State,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Policies,Policy Type,Policy,Renew Offer Type,Total Claim Amount,Vehicle Class,Vehicle Size,Customer Lifetime Value
0,Washington,Basic,Bachelor,2/24/2011,Employed,F,56274,Suburban,Married,69,32,5,1,Corporate Auto,Corporate L3,Offer1,384.811147,Two-Door Car,Medsize,2763.519279
1,Arizona,Extended,Bachelor,1/31/2011,Unemployed,F,0,Suburban,Single,94,13,42,8,Personal Auto,Personal L3,Offer3,1131.464935,Four-Door Car,Medsize,6979.535903
2,Nevada,Premium,Bachelor,2/19/2011,Employed,F,48767,Suburban,Married,108,18,38,2,Personal Auto,Personal L3,Offer1,566.472247,Two-Door Car,Medsize,12887.431650
3,California,Basic,Bachelor,1/20/2011,Unemployed,M,0,Suburban,Married,106,18,65,7,Corporate Auto,Corporate L2,Offer1,529.881344,SUV,Medsize,7645.861827
4,Washington,Basic,Bachelor,2/3/2011,Employed,M,43836,Rural,Single,73,12,44,1,Personal Auto,Personal L1,Offer1,138.130879,Four-Door Car,Medsize,2813.692575


In [86]:
np.shape(df)

(9134, 20)

In [87]:
df.corr()

,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Policies,Total Claim Amount,Customer Lifetime Value
Income,1.000000,-0.016665,-0.026715,-0.000875,-0.008656,-0.355254,0.024366
Monthly Premium Auto,-0.016665,1.000000,0.005026,0.020257,-0.011233,0.632017,0.396262
Months Since Last Claim,-0.026715,0.005026,1.000000,-0.042959,0.009136,0.007563,0.011517
Months Since Policy Inception,-0.000875,0.020257,-0.042959,1.000000,-0.013333,0.003335,0.009418
Number of Policies,-0.008656,-0.011233,0.009136,-0.013333,1.000000,-0.002354,0.021955
Total Claim Amount,-0.355254,0.632017,0.007563,0.003335,-0.002354,1.000000,0.226451
Customer Lifetime Value,0.024366,0.396262,0.011517,0.009418,0.021955,0.226451,1.000000


In [88]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Income,9134.0,37657.380009,30379.904734,0.000000,0.000000,33889.500000,62320.000000,99981.000000
Monthly Premium Auto,9134.0,93.219291,34.407967,61.000000,68.000000,83.000000,109.000000,298.000000
Months Since Last Claim,9134.0,15.097000,10.073257,0.000000,6.000000,14.000000,23.000000,35.000000
Months Since Policy Inception,9134.0,48.064594,27.905991,0.000000,24.000000,48.000000,71.000000,99.000000
Number of Policies,9134.0,2.966170,2.390182,1.000000,1.000000,2.000000,4.000000,9.000000
Total Claim Amount,9134.0,434.088794,290.500092,0.099007,272.258244,383.945434,547.514839,2893.239678
Customer Lifetime Value,9134.0,8004.940475,6870.967608,1898.007675,3994.251794,5780.182197,8962.167041,83325.381190


### Labeling the categorical variables: 

In [97]:
df["Education"] = df["Education"].astype('category')
df["State"] = df["State"].astype('category')

df["EmploymentStatus"] = df["EmploymentStatus"].astype('category')
df["Location Code"] = df["Location Code"].astype('category')

df["Marital Status"] = df["Marital Status"].astype('category')

df["Gender"] = df["Gender"].astype('category')
df["Policy Type"] = df["Policy Type"].astype('category')
df["Policy"] = df["Policy"].astype('category')


df["Renew Offer Type"] = df["Renew Offer Type"].astype('category')
df["Vehicle Class"] = df["Vehicle Class"].astype('category')
df["Vehicle Size"] = df["Vehicle Size"].astype('category')

df.dtypes

State                            category
Coverage                           object
Education                        category
Effective To Date                  object
EmploymentStatus                 category
Gender                           category
Income                              int64
Location Code                    category
Marital Status                   category
Monthly Premium Auto                int64
Months Since Last Claim             int64
Months Since Policy Inception       int64
Number of Policies                  int64
Policy Type                      category
Policy                           category
Renew Offer Type                 category
Total Claim Amount                float64
Vehicle Class                    category
Vehicle Size                     category
Customer Lifetime Value           float64
Marital Status_cat               category
dtype: object

In [98]:
df["Education_cat"] = df["Education"].cat.codes

df["State_cat"] = df["State"].cat.codes

df["EmploymentStatus_cat"] = df["EmploymentStatus"].cat.codes
df["Location Code_cat"] = df["Location Code"].cat.codes

df["Marital Status_cat"] = df["Marital Status"].cat.codes

df["Gender_cat"] = df["Gender"].cat.codes
df["Policy Type_cat"] = df["Policy Type"].cat.codes
df["Policy_cat"] = df["Policy"].cat.codes


df["Renew Offer Type_cat"] = df["Renew Offer Type"].cat.codes
df["Vehicle Class_cat"] = df["Vehicle Class"].cat.codes
df["Vehicle Size_cat"] = df["Vehicle Size"].cat.codes

df.head()


,State,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,...,Education_cat,State_cat,EmploymentStatus_cat,Location Code_cat,Gender_cat,Policy Type_cat,Policy_cat,Renew Offer Type_cat,Vehicle Class_cat,Vehicle Size_cat
0,Washington,Basic,Bachelor,2/24/2011,Employed,F,56274,Suburban,Married,69,...,0,4,1,1,0,0,2,0,5,1
1,Arizona,Extended,Bachelor,1/31/2011,Unemployed,F,0,Suburban,Single,94,...,0,0,4,1,0,1,5,2,0,1
2,Nevada,Premium,Bachelor,2/19/2011,Employed,F,48767,Suburban,Married,108,...,0,2,1,1,0,1,5,0,5,1
3,California,Basic,Bachelor,1/20/2011,Unemployed,M,0,Suburban,Married,106,...,0,1,4,1,1,0,1,0,3,1
4,Washington,Basic,Bachelor,2/3/2011,Employed,M,43836,Rural,Single,73,...,0,4,1,0,1,1,3,0,0,1


In [99]:
train, test = train_test_split(df, test_size=0.2)

In [100]:
train.corr()

,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Policies,Total Claim Amount,Customer Lifetime Value,Marital Status_cat,Education_cat,State_cat,EmploymentStatus_cat,Location Code_cat,Gender_cat,Policy Type_cat,Policy_cat,Renew Offer Type_cat,Vehicle Class_cat,Vehicle Size_cat
Income,1.000000,-0.004099,-0.031537,-0.000901,0.000571,-0.348304,0.026984,-0.232334,0.015879,-0.004270,-0.727108,-0.027050,-0.010080,-0.000781,-0.001333,0.141234,0.011031,0.011137
Monthly Premium Auto,-0.004099,1.000000,0.000463,0.025183,-0.004524,0.627941,0.404517,0.010056,0.020953,0.006118,0.000687,0.007075,0.011137,0.004493,0.000743,-0.070547,0.172032,0.003619
Months Since Last Claim,-0.031537,0.000463,1.000000,-0.042382,0.015762,0.007221,0.009003,0.004602,0.001909,0.011900,0.011573,-0.017578,0.027771,-0.007454,-0.004097,-0.015363,-0.007231,-0.003209
Months Since Policy Inception,-0.000901,0.025183,-0.042382,1.000000,-0.014215,0.002705,0.009930,-0.033363,0.001305,0.008525,0.010751,0.000983,0.005691,-0.005136,-0.008239,-0.094310,0.002391,0.010551
Number of Policies,0.000571,-0.004524,0.015762,-0.014215,1.000000,-0.005933,0.027325,0.007668,0.011755,-0.018554,0.001169,-0.005635,0.020130,0.003637,0.001626,-0.082377,0.000597,0.010982
Total Claim Amount,-0.348304,0.627941,0.007221,0.002705,-0.005933,1.000000,0.228942,0.200309,0.014626,0.009188,0.326529,0.260341,0.071585,0.010591,0.003123,-0.080196,0.105548,0.070391
Customer Lifetime Value,0.026984,0.404517,0.009003,0.009930,0.027325,0.228942,1.000000,-0.026975,0.023763,0.002041,-0.038075,0.003346,-0.013679,0.017485,0.009202,-0.080369,0.083023,0.012020
Marital Status_cat,-0.232334,0.010056,0.004602,-0.033363,0.007668,0.200309,-0.026975,1.000000,0.010195,0.018009,0.313161,0.012499,0.027657,0.009724,0.003793,-0.027319,0.023208,0.008248
Education_cat,0.015879,0.020953,0.001909,0.001305,0.011755,0.014626,0.023763,0.010195,1.000000,0.019454,-0.026000,-0.016470,-0.009830,0.008588,0.008580,0.067385,0.005793,-0.001890
State_cat,-0.004270,0.006118,0.011900,0.008525,-0.018554,0.009188,0.002041,0.018009,0.019454,1.000000,0.004959,-0.004063,-0.012150,0.006630,0.009234,-0.003406,0.012754,0.003629


### ANN Model for Regression:

In [101]:
# x = train[['Total Claim Amount', 'Income', 'Education_cat', 'Months Since Last Claim', 'State_cat', 'Gender_cat', 'EmploymentStatus_cat', 'Location Code_cat', 'Gender_cat']]
# y = train['Customer Lifetime Value']

x = train[['Marital Status_cat']]
y = train['Customer Lifetime Value']

In [102]:
x.head()

,Marital Status_cat
4003,0
5710,2
2542,2
3169,1
6421,2


In [103]:
n =  np.shape(train)[0]
# x = np.random.uniform(-15, 15, size = n)
# y = x**2 + 2*np.random.randn(n, )
X = np.reshape(x ,[n, 1]) 
y = np.reshape(y ,[n ,])

C:\Users\e84120074\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:58: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return bound(*args, **kwds)


In [104]:
clf = MLPRegressor(alpha=0.01, hidden_layer_sizes = (5,), max_iter = 50000, 
                 activation = 'logistic', verbose = 'True', learning_rate = 'adaptive')
a = clf.fit(X, y)

Iteration 1, loss = 54677673.76675989
Iteration 2, loss = 54676511.24850037
Iteration 3, loss = 54675320.30374575
Iteration 4, loss = 54674094.39152899
Iteration 5, loss = 54672829.27225518
Iteration 6, loss = 54671521.87918242
Iteration 7, loss = 54670171.20282885
Iteration 8, loss = 54668773.09623872
Iteration 9, loss = 54667332.64587353
Iteration 10, loss = 54665848.37331437
Iteration 11, loss = 54664321.17965838
Iteration 12, loss = 54662749.37141240
Iteration 13, loss = 54661138.29371364
Iteration 14, loss = 54659485.46485785
Iteration 15, loss = 54657782.84181825
Iteration 16, loss = 54656039.43250624
Iteration 17, loss = 54654242.02563987
Iteration 18, loss = 54652390.04640121
Iteration 19, loss = 54650485.37459549
Iteration 20, loss = 54648530.04238287
Iteration 21, loss = 54646527.25014456
Iteration 22, loss = 54644491.48303817
Iteration 23, loss = 54642410.22989362
Iteration 24, loss = 54640312.12774178
Iteration 25, loss = 54638190.81939691
Iteration 26, loss = 54636047.6235

Iteration 217, loss = 54278867.25457405
Iteration 218, loss = 54277106.89648654
Iteration 219, loss = 54275351.53445677
Iteration 220, loss = 54273597.33443289
Iteration 221, loss = 54271837.77162569
Iteration 222, loss = 54270084.23152232
Iteration 223, loss = 54268327.79316172
Iteration 224, loss = 54266573.76912025
Iteration 225, loss = 54264820.38179420
Iteration 226, loss = 54263059.39031298
Iteration 227, loss = 54261309.10066039
Iteration 228, loss = 54259552.84319281
Iteration 229, loss = 54257799.11930586
Iteration 230, loss = 54256047.01522491
Iteration 231, loss = 54254289.20409370
Iteration 232, loss = 54252540.67284093
Iteration 233, loss = 54250784.41673631
Iteration 234, loss = 54249034.05356227
Iteration 235, loss = 54247277.08152122
Iteration 236, loss = 54245521.16522680
Iteration 237, loss = 54243768.45315782
Iteration 238, loss = 54242014.74551030
Iteration 239, loss = 54240256.88546751
Iteration 240, loss = 54238499.41643574
Iteration 241, loss = 54236751.25180418


Iteration 428, loss = 53910254.92301875
Iteration 429, loss = 53908518.96066821
Iteration 430, loss = 53906777.26574787
Iteration 431, loss = 53905035.68494272
Iteration 432, loss = 53903295.81827870
Iteration 433, loss = 53901554.64658408
Iteration 434, loss = 53899816.89008763
Iteration 435, loss = 53898074.67047646
Iteration 436, loss = 53896332.48658832
Iteration 437, loss = 53894594.15455189
Iteration 438, loss = 53892857.49884697
Iteration 439, loss = 53891112.42572771
Iteration 440, loss = 53889375.55949119
Iteration 441, loss = 53887632.15573186
Iteration 442, loss = 53885895.09624486
Iteration 443, loss = 53884155.24937776
Iteration 444, loss = 53882418.87940396
Iteration 445, loss = 53880680.90450204
Iteration 446, loss = 53878940.27904090
Iteration 447, loss = 53877202.80210801
Iteration 448, loss = 53875462.42549065
Iteration 449, loss = 53873724.34575789
Iteration 450, loss = 53871986.56765009
Iteration 451, loss = 53870247.89685354
Iteration 452, loss = 53868508.64858287


Iteration 642, loss = 53538978.73687690
Iteration 643, loss = 53537250.23952291
Iteration 644, loss = 53535516.41136924
Iteration 645, loss = 53533786.81333281
Iteration 646, loss = 53532060.18370659
Iteration 647, loss = 53530328.86987877
Iteration 648, loss = 53528601.55296590
Iteration 649, loss = 53526868.34387325
Iteration 650, loss = 53525141.10757539
Iteration 651, loss = 53523406.98851176
Iteration 652, loss = 53521679.50905366
Iteration 653, loss = 53519950.56829276
Iteration 654, loss = 53518222.48376736
Iteration 655, loss = 53516492.08044974
Iteration 656, loss = 53514765.13257303
Iteration 657, loss = 53513037.67073145
Iteration 658, loss = 53511303.23227532
Iteration 659, loss = 53509578.32790720
Iteration 660, loss = 53507848.21735985
Iteration 661, loss = 53506117.53540789
Iteration 662, loss = 53504389.20036183
Iteration 663, loss = 53502659.87844358
Iteration 664, loss = 53500930.76158371
Iteration 665, loss = 53499202.57971460
Iteration 666, loss = 53497474.67412717


Iteration 852, loss = 53176854.45651678
Iteration 853, loss = 53175134.43322488
Iteration 854, loss = 53173418.73475054
Iteration 855, loss = 53171699.00082687
Iteration 856, loss = 53169979.00198105
Iteration 857, loss = 53168263.92339621
Iteration 858, loss = 53166543.90325136
Iteration 859, loss = 53164823.15827646
Iteration 860, loss = 53163104.94025060
Iteration 861, loss = 53161386.45114261
Iteration 862, loss = 53159666.61076149
Iteration 863, loss = 53157949.66129530
Iteration 864, loss = 53156233.38072390
Iteration 865, loss = 53154511.23056266
Iteration 866, loss = 53152796.68674627
Iteration 867, loss = 53151080.26639162
Iteration 868, loss = 53149360.81206221
Iteration 869, loss = 53147642.16831779
Iteration 870, loss = 53145923.13592796
Iteration 871, loss = 53144203.99327859
Iteration 872, loss = 53142486.76000530
Iteration 873, loss = 53140767.06440365
Iteration 874, loss = 53139050.02447343
Iteration 875, loss = 53137328.70738303
Iteration 876, loss = 53135615.19935611


...

Iteration 40253, loss = 23034132.15673274
Iteration 40254, loss = 23034131.59117395
Iteration 40255, loss = 23034130.43850767
Iteration 40256, loss = 23034129.97885612
Iteration 40257, loss = 23034130.30861444
Iteration 40258, loss = 23034127.70507941
Iteration 40259, loss = 23034126.75293511
Iteration 40260, loss = 23034125.60426719
Iteration 40261, loss = 23034124.98882652
Iteration 40262, loss = 23034124.29326888
Iteration 40263, loss = 23034123.86275021
Iteration 40264, loss = 23034122.60507261
Iteration 40265, loss = 23034122.10582341
Iteration 40266, loss = 23034120.60913878
Iteration 40267, loss = 23034119.48600920
Iteration 40268, loss = 23034119.33339162
Iteration 40269, loss = 23034117.43079236
Iteration 40270, loss = 23034118.09423674
Iteration 40271, loss = 23034115.56504362
Iteration 40272, loss = 23034114.37706050
Iteration 40273, loss = 23034113.38135732
Iteration 40274, loss = 23034112.47311219
Iteration 40275, loss = 23034111.94466235
Iteration 40276, loss = 23034111.4

Iteration 40454, loss = 23033954.80432336
Iteration 40455, loss = 23033954.95768668
Iteration 40456, loss = 23033953.01500576
Iteration 40457, loss = 23033952.17719174
Iteration 40458, loss = 23033952.26514200
Iteration 40459, loss = 23033951.17157517
Iteration 40460, loss = 23033950.77764709
Iteration 40461, loss = 23033949.61395805
Iteration 40462, loss = 23033949.78485347
Iteration 40463, loss = 23033947.80782663
Iteration 40464, loss = 23033947.14300383
Iteration 40465, loss = 23033946.59652033
Iteration 40466, loss = 23033945.64029382
Iteration 40467, loss = 23033944.45262011
Iteration 40468, loss = 23033943.16719246
Iteration 40469, loss = 23033942.83937070
Iteration 40470, loss = 23033942.30997509
Iteration 40471, loss = 23033941.06405899
Iteration 40472, loss = 23033940.25375368
Iteration 40473, loss = 23033939.46557177
Iteration 40474, loss = 23033940.08769185
Iteration 40475, loss = 23033937.71875723
Iteration 40476, loss = 23033936.65555211
Iteration 40477, loss = 23033935.7

Iteration 40657, loss = 23033809.33155231
Iteration 40658, loss = 23033809.35362640
Iteration 40659, loss = 23033808.48171743
Iteration 40660, loss = 23033807.24851142
Iteration 40661, loss = 23033808.63749222
Iteration 40662, loss = 23033806.35259527
Iteration 40663, loss = 23033805.58889636
Iteration 40664, loss = 23033804.98750440
Iteration 40665, loss = 23033806.73467087
Iteration 40666, loss = 23033805.55808239
Iteration 40667, loss = 23033802.54653765
Iteration 40668, loss = 23033802.30779761
Iteration 40669, loss = 23033802.01387961
Iteration 40670, loss = 23033802.18584819
Iteration 40671, loss = 23033800.63883843
Iteration 40672, loss = 23033799.79870469
Iteration 40673, loss = 23033799.59286658
Iteration 40674, loss = 23033798.51703108
Iteration 40675, loss = 23033798.21713589
Iteration 40676, loss = 23033797.02826357
Iteration 40677, loss = 23033796.19831271
Iteration 40678, loss = 23033795.39610482
Iteration 40679, loss = 23033795.32198017
Iteration 40680, loss = 23033794.3

Iteration 40858, loss = 23033690.17164425
Iteration 40859, loss = 23033687.76088057
Iteration 40860, loss = 23033687.05621553
Iteration 40861, loss = 23033687.16224999
Iteration 40862, loss = 23033686.87491696
Iteration 40863, loss = 23033685.75498570
Iteration 40864, loss = 23033685.92158642
Iteration 40865, loss = 23033685.76465945
Iteration 40866, loss = 23033684.08786945
Iteration 40867, loss = 23033683.94501269
Iteration 40868, loss = 23033682.27199645
Iteration 40869, loss = 23033682.11900068
Iteration 40870, loss = 23033682.73148870
Iteration 40871, loss = 23033681.23244384
Iteration 40872, loss = 23033680.86404426
Iteration 40873, loss = 23033679.96861536
Iteration 40874, loss = 23033679.90688899
Iteration 40875, loss = 23033678.57300654
Iteration 40876, loss = 23033678.71461767
Iteration 40877, loss = 23033678.33303120
Iteration 40878, loss = 23033677.44563442
Iteration 40879, loss = 23033677.50982680
Iteration 40880, loss = 23033676.22759327
Iteration 40881, loss = 23033675.5

Iteration 41076, loss = 23033581.38473109
Iteration 41077, loss = 23033580.90441273
Iteration 41078, loss = 23033580.70594700
Iteration 41079, loss = 23033580.98602337
Iteration 41080, loss = 23033579.91937996
Iteration 41081, loss = 23033579.55873154
Iteration 41082, loss = 23033580.47271175
Iteration 41083, loss = 23033579.40380321
Iteration 41084, loss = 23033578.75461658
Iteration 41085, loss = 23033578.54582122
Iteration 41086, loss = 23033578.48447241
Iteration 41087, loss = 23033577.54067208
Iteration 41088, loss = 23033580.55588720
Iteration 41089, loss = 23033576.64089958
Iteration 41090, loss = 23033576.12738620
Iteration 41091, loss = 23033575.51163025
Iteration 41092, loss = 23033574.89195197
Iteration 41093, loss = 23033574.97026589
Iteration 41094, loss = 23033573.96898959
Iteration 41095, loss = 23033574.36506248
Iteration 41096, loss = 23033574.49793098
Iteration 41097, loss = 23033572.64711463
Iteration 41098, loss = 23033574.52075867
Iteration 41099, loss = 23033571.8

In [105]:
#x_ = np.linspace(-4, 4, 160) # define axis

pred_x = np.reshape(test[['Gender_cat']], [len(test), 1]) # [160, ] -> [160, 1]
pred_y = clf.predict(pred_x) # predict network output given x_
e = np.square(np.subtract(test['Income'],pred_y)).mean()
e
# fig = plt.figure() 
# plt.plot(e, color = 'b') # plot original function
# plt.plot(pred_y, test['Customer Lifetime Value'], '-') # plot network output

1820559826.928394

In [106]:
# print("w0 = ", clf.coefs_[0][0][0])
# print("w1 = ", clf.coefs_[0][1][0])

e1 >> Single input variable >> 43846984.98455383

e2 >> only two variables as input >> 43845846.135405615  (lower error)

e3 >> all numerical variables as input >> 49912090.82538457 

e4 >> settings are the same as e3 .. but with less number of hidden layers >> 50057390.290026695 (the highest error)  .. 


Now, let's include some other categorical variables that might enhance the performance of the model (but after encoding them) ..

Let's start with "Label encoding" ..

In [107]:
print("weights between input and first hidden layer:")
print(clf.coefs_[0])
print("\nweights between first hidden and second hidden layer:")
print(clf.coefs_[1])

weights between input and first hidden layer:
[[ 1.65540860e-316  1.99452760e-316 -2.23767163e-316  7.66650457e-317
  -3.19313377e-316]]

weights between first hidden and second hidden layer:
[[1321.50979584]
 [1321.11025842]
 [1321.56268113]
 [1320.63141632]
 [1320.97871489]]


In [108]:
np.mean(clf.coefs_[1])

1321.1585733201873

Consequently .. 
we gonna use these weights as coefficient of each variable in the system dynamics models 

### ANN for Classification:

In [93]:
x = train[['Income']]
y = train['State_cat']

In [94]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, ))

In [95]:
clf.fit(x, y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [96]:
pred_x = np.reshape(test[['Income']], [len(test), 1]) # [160, ] -> [160, 1]
pred_y = clf.predict(pred_x) # predict network output given x_

In [97]:
print("weights between input and first hidden layer:")
print(clf.coefs_[0])
print("\nweights between first hidden and second hidden layer:")
print(clf.coefs_[1])

weights between input and first hidden layer:
[[-0.47239889 -0.91509192 -0.88466448 -0.18851828 -0.46220396]]

weights between first hidden and second hidden layer:
[[ 0.04495002  0.54735497  0.13629906 -0.61392641  0.67536616]
 [-0.68019516 -0.37502159  0.62497645  0.14427496 -0.61729612]
 [-0.09645186 -0.6345569  -0.58010825  0.57416082  0.62870464]
 [ 0.26063157  0.64782431  0.66653101  0.06332955  0.0746983 ]
 [-0.32620795  0.77052149  0.23369004 -0.23780003 -0.45099275]]


In [98]:
np.mean(clf.coefs_[1])

0.05923025305326732